In [ ]:
import os
import pandas as pd
import numpy as np
import wfdb
import matplotlib.pyplot as plt 

In [ ]:
!pip install scipy

In [ ]:
!pip install seaborn

In [ ]:
# Define the data directory
data_dir = r"C:\Users\User\Document\autonomic-aging-a-dataset-to-quantify-changes-of-cardiovascular-autonomic-function-during-healthy-aging-1.0.0"

# Load clinical data
clinical_data_path = os.path.join(data_dir, 'subject-info.csv')
clinical_data = pd.read_csv(clinical_data_path)

# Preview clinical data
print(clinical_data.head())

In [ ]:
# Define data folder
data_folder = r"C:\Users\User\Document\autonomic-aging-a-dataset-to-quantify-changes-of-cardiovascular-autonomic-function-during-healthy-aging-1.0.0"

# Preview .hea and .dat Files
def preview_files(folder_path):
    print("\nPreviewing .hea and .dat files:")
    # List all .hea and .dat files in the specified directory
    for filename in os.listdir(folder_path):
        if filename.endswith('.hea'):
            record_name = os.path.join(folder_path, filename[:-4])  # Remove the '.hea' extension
            # Read the record using wfdb
            record = wfdb.rdrecord(record_name)
            print(f"\nHeader information for {filename}:")
            print(f"Signal names: {record.sig_name}")
            print(f"Number of signals: {record.n_sig}")
            print(f"Sampling frequency: {record.fs} Hz")
            print(f"Data shape: {record.p_signal.shape}")
        elif filename.endswith('.dat'):
            record_name = os.path.join(folder_path, filename[:-4])  # Remove the '.dat' extension
            # Read the record using wfdb
            record = wfdb.rdrecord(record_name)
            print(f"\nData preview for {filename}:")
            print(record.p_signal[:5])  # Print first few samples of signal data

# Call the function to preview .hea and .dat files
preview_files(data_folder)


In [ ]:
import os
import wfdb
import matplotlib.pyplot as plt

# Define data folder
data_folder = r"C:\Users\User\Document\autonomic-aging-a-dataset-to-quantify-changes-of-cardiovascular-autonomic-function-during-healthy-aging-1.0.0"

# Function to preview and plot signals from .hea and .dat files
def preview_and_plot_signals(folder_path, record_id):
    print(f"\nPreviewing and plotting signals for record {record_id}:")
    record_name = os.path.join(folder_path, record_id)  # Construct the full path without extension

    # Read the record using wfdb
    record = wfdb.rdrecord(record_name)

    # Print header information
    print(f"\nHeader information for {record_id}.hea:")
    print(f"Signal names: {record.sig_name}")
    print(f"Number of signals: {record.n_sig}")
    print(f"Sampling frequency: {record.fs} Hz")
    print(f"Data shape: {record.p_signal.shape}")

    # Preview first few samples of signal data
    print(f"\nData preview for {record_id}.dat:")
    print(record.p_signal[:5])  # Print first few samples of signal data

    # Plot each signal
    for i, signal_name in enumerate(record.sig_name):
        plt.figure(figsize=(10, 4))
        plt.plot(record.p_signal[:, i], label=signal_name)
        plt.title(f"{signal_name} Distribution for {record_id}")
        plt.xlabel('Sample Index')
        plt.ylabel('Amplitude')
        plt.legend()
        plt.grid(True)
        plt.show()

# Call the function to preview and plot signals for sample 1
preview_and_plot_signals(data_folder, '0001')

In [ ]:
import wfdb
import numpy as np
import matplotlib.pyplot as plt

# Load record 0001
record = wfdb.rdrecord('0001', pn_dir='autonomic-aging-cardiovascular/1.0.0/')
signals = record.p_signal
sampling_frequency = record.fs

# Extract ECG signal (assuming it's the first signal)
ecg_signal = signals[:, 0]

# Time vector for the ECG signal
time = np.arange(len(ecg_signal)) / sampling_frequency

# Define the duration to display (2.5 seconds)
duration = 2.5
start_index = 0  # Start from the beginning of the signal
end_index = int(duration * sampling_frequency)  # Calculate the end index for 2.5 seconds

# Plotting the ECG signal for the specified duration
plt.figure(figsize=(12, 6))
plt.plot(time[start_index:end_index], ecg_signal[start_index:end_index], color='blue', linewidth=1.5)
plt.title('ECG Signal (First 2.5 Seconds)')
plt.xlabel('Time (seconds)')
plt.ylabel('Amplitude')
plt.xlim([0, duration])  # Set x-axis limit to 2.5 seconds
plt.grid()
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
from scipy.signal import butter, lfilter

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='bandpass')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

# Apply a bandpass filter (0.5 Hz - 40 Hz)
filtered_ecg = butter_bandpass_filter(ecg_signal, 1, 50, sampling_frequency)




In [ ]:
import wfdb
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks, butter, lfilter

# Load record 0001
record = wfdb.rdrecord('0001', pn_dir='autonomic-aging-cardiovascular/1.0.0/')
signals = record.p_signal
sampling_frequency = record.fs

# Extract ECG signal (assuming it's the first signal)
ecg_signal = signals[:, 0]

# Bandpass filter functions
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='bandpass')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

# Apply a bandpass filter (1 Hz - 50 Hz)
filtered_ecg = butter_bandpass_filter(ecg_signal, 1, 50, sampling_frequency)

# Simple R-peak detection function
def detect_r_peaks(ecg_signal, sampling_rate, threshold=0.5):
    """Simple R-peak detection based on thresholding."""
    peaks, _ = find_peaks(ecg_signal, height=threshold)
    return peaks

# Detect R-peaks using the filtered ECG signal
r_peaks = detect_r_peaks(filtered_ecg, sampling_frequency)

# Time vector for the filtered ECG signal
time = np.arange(len(filtered_ecg)) / sampling_frequency

# Plotting the filtered ECG signal with R-peaks
plt.figure(figsize=(12, 6))
plt.plot(time, filtered_ecg, color='blue', linewidth=1.5, label='Filtered ECG Signal')
plt.plot(time[r_peaks], filtered_ecg[r_peaks], 'ro', label='R-peaks', markersize=5)
plt.title('Filtered ECG Signal with R-Peaks', fontsize=16)
plt.xlabel('Time (seconds)', fontsize=14)
plt.ylabel('Amplitude', fontsize=14)
plt.grid()
plt.legend()
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

# Assuming filtered_ecg and sampling_frequency are already defined

# Detect R-peaks using the filtered ECG signal
r_peaks = find_peaks(filtered_ecg, height=0.5)[0]

# Calculate RR intervals in seconds
rr_intervals_seconds = np.diff(r_peaks) / sampling_frequency

# Print RR intervals
print("RR intervals (seconds):", rr_intervals_seconds)

# Plotting the RR intervals
plt.figure(figsize=(12, 6))
plt.bar(range(len(rr_intervals_seconds)), rr_intervals_seconds, color='orange')
plt.title('RR Intervals', fontsize=16)
plt.xlabel('Interval Number', fontsize=14)
plt.ylabel('Duration (seconds)', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(axis='y')
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
# Calculate heart rate in BPM for each RR interval
heart_rates = 60 / rr_intervals_seconds

print("Heart rates (BPM):", heart_rates)


In [ ]:
import wfdb
import numpy as np
from scipy.signal import butter, lfilter, find_peaks
import matplotlib.pyplot as plt

In [ ]:
import wfdb
import numpy as np
from scipy.signal import butter, lfilter, find_peaks

# Section 1: Define Filtering Functions
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='bandpass')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

    # Section 2: Define R-Peak Detection and Heart Rate Calculation Functions
    def detect_r_peaks(ecg_signal, sampling_rate, threshold = 0.5):
        """Simple R-peak detection based on thresholding."""
        peaks, _ = find_peaks(ecg_signal, height = threshold)
        return peaks

def calculate_heart_rate(rr_intervals_seconds):
    return 60 / rr_intervals_seconds

def calculate_hrv(rr_intervals_seconds):
    # Simple HRV calculation: Standard Deviation of RR intervals (SDNN)
    return np.std(rr_intervals_seconds)


In [ ]:
# Section 3: Load Data and Process (First Half)
import wfdb
import numpy as np
import pandas as pd

# Define your functions like butter_bandpass_filter, detect_r_peaks, calculate_heart_rate, calculate_hrv here

# Sample files
sample_files = [f"{i:04d}" for i in range(1, 1121)]  # Adjust range as needed
first_half_files = sample_files[:605]  # First half of files

# Initialize lists to store results
results_first_half = []

for i, file in enumerate(first_half_files):
    try:
        record = wfdb.rdrecord(file, pn_dir='autonomic-aging-cardiovascular/1.0.0/')
        signals = record.p_signal
        sampling_frequency = record.fs
        ecg_signal = signals[:, 0]  # Assuming ECG is the first signal
        
        # Apply a bandpass filter (1 Hz - 50 Hz)
        filtered_ecg = butter_bandpass_filter(ecg_signal, 2, 40, sampling_frequency)
        
        # Detect R-peaks
        r_peaks = detect_r_peaks(filtered_ecg, sampling_frequency)
        
        if len(r_peaks) > 1:  # Ensure there are at least two peaks for RR interval calculation
            # Calculate RR intervals in samples
            rr_intervals_samples = np.diff(r_peaks)
            
            # Convert RR intervals to seconds
            rr_intervals_seconds = rr_intervals_samples / sampling_frequency
            
            # Calculate heart rate in BPM for each RR interval
            heart_rates = calculate_heart_rate(rr_intervals_seconds)
            
            # Calculate mean heart rate
            mean_heart_rate = np.mean(heart_rates)
            
            # Calculate standard deviation of heart rate
            std_heart_rate = np.std(heart_rates)
            
            # Calculate HRV (SDNN)
            hrv = calculate_hrv(rr_intervals_seconds)
            
            # Store results
            results_first_half.append({
                "File": file,
                "Mean Heart Rate": mean_heart_rate,
                "Std Heart Rate": std_heart_rate,
                "HRV": hrv,
                "R-peaks": r_peaks,
                "RR Intervals (s)": rr_intervals_seconds,
                "Heart Rates (BPM)": heart_rates
            })
            
            # Print detailed results for the last sample every 50 processed
            if (i + 1) % 50 == 0:
                result = results_first_half[-1]  # Get the last result of the processed batch
                print(f"Processed {i + 1} files in the first half.")
                print(f"File: {result['File']}")
                print(f"R-peaks: {result['R-peaks']}")
                print(f"RR Intervals (s): {result['RR Intervals (s)']}")
                print(f"Heart Rates (BPM): {result['Heart Rates (BPM)']}")
                print(f"Mean Heart Rate: {result['Mean Heart Rate']}, Std Heart Rate: {result['Std Heart Rate']}, HRV: {result['HRV']}")
                print("-" * 50)

    except Exception as e:
        print(f"Error processing file {file}: {e}")

# Convert results to DataFrame for future analysis
df_first_half = pd.DataFrame(results_first_half)

# Save results to CSV (optional)
df_first_half.to_csv('heart_rate_analysis_first_half.csv', index=False)

In [ ]:
# Section 3: Load Data and Process (Second Half)
# Sample files (second half)
second_half_files = sample_files[605:]  # Second half of files

# Initialize lists to store results for the second half
results_second_half = []

for i, file in enumerate(second_half_files):
    try:
        record = wfdb.rdrecord(file, pn_dir='autonomic-aging-cardiovascular/1.0.0/')
        signals = record.p_signal
        sampling_frequency = record.fs
        ecg_signal = signals[:, 0]  # Assuming ECG is the first signal
        
        # Apply a bandpass filter (1 Hz - 50 Hz)
        filtered_ecg = butter_bandpass_filter(ecg_signal, 2, 40, sampling_frequency)
        
        # Detect R-peaks
        r_peaks = detect_r_peaks(filtered_ecg, sampling_frequency)
        
        if len(r_peaks) > 1:  # Ensure there are at least two peaks for RR interval calculation
            # Calculate RR intervals in samples
            rr_intervals_samples = np.diff(r_peaks)
            
            # Convert RR intervals to seconds
            rr_intervals_seconds = rr_intervals_samples / sampling_frequency
            
            # Calculate heart rate in BPM for each RR interval
            heart_rates = calculate_heart_rate(rr_intervals_seconds)
            
            # Calculate mean heart rate
            mean_heart_rate = np.mean(heart_rates)
            
            # Calculate standard deviation of heart rate
            std_heart_rate = np.std(heart_rates)
            
            # Calculate HRV (SDNN)
            hrv = calculate_hrv(rr_intervals_seconds)
            
            # Store results
            results_second_half.append({
                "File": file,
                "Mean Heart Rate": mean_heart_rate,
                "Std Heart Rate": std_heart_rate,
                "HRV": hrv,
                "R-peaks": r_peaks,
                "RR Intervals (s)": rr_intervals_seconds,
                "Heart Rates (BPM)": heart_rates
            })
            
            # Print detailed results for the last sample every 50 processed
            if (i + 1) % 50 == 0:
                result = results_second_half[-1]  # Get the last result of the processed batch
                print(f"Processed {i + 1} files in the second half.")
                print(f"File: {result['File']}")
                print(f"R-peaks: {result['R-peaks']}")
                print(f"RR Intervals (s): {result['RR Intervals (s)']}")
                print(f"Heart Rates (BPM): {result['Heart Rates (BPM)']}")
                print(f"Mean Heart Rate: {result['Mean Heart Rate']}, Std Heart Rate: {result['Std Heart Rate']}, HRV: {result['HRV']}")
                print("-" * 50)

    except Exception as e:
        print(f"Error processing file {file}: {e}")

# Convert results to DataFrame for future analysis
df_second_half = pd.DataFrame(results_second_half)

# Save results to CSV (optional)
df_second_half.to_csv('heart_rate_analysis_second_half.csv', index=False)

# Combine the results from both halves for further analysis
combined_results = pd.concat([df_first_half, df_second_half], ignore_index=True)

# Optionally save combined results
combined_results.to_csv('heart_rate_analysis_combined.csv', index=False)

In [ ]:
# Load the combined results
combined_results = pd.read_csv('heart_rate_analysis_combined.csv')

# Merge with clinical data
merged_data = combined_results.merge(clinical_data, left_on='File', right_on='ID', how='left')


# Reset the index to start from 1
merged_data.reset_index(drop=True, inplace=True)
merged_data.index += 1  # Shift index to start from 1

# Preview the merged data with the new index
print(merged_data.head())

In [ ]:
# Remove samples with age_group = NaN
merged_data = merged_data[merged_data['Age_group'].notna()]

# Remove samples with Mean Heart Rate larger than 150 or smaller than 10
merged_data = merged_data[(merged_data['Mean Heart Rate'] <= 150) & (merged_data['Mean Heart Rate'] >= 10)]

# Keep data with HRV between 0.01 and 0.1
merged_data = merged_data[(merged_data['HRV'] >= 0.01) & (merged_data['HRV'] <= 0.1)]

# Reset the index again after filtering
merged_data.reset_index(drop=True, inplace=True)
merged_data.index += 1  # Shift index to start from 1

# Preview the filtered merged data
print(merged_data.head())

In [ ]:
print(merged_data['Sex'].unique())

In [ ]:
# Filter data for males and females based on your dataset
males_data = merged_data[merged_data['Sex'] == 0]  # Males are 1
females_data = merged_data[merged_data['Sex'] == 1]  # Females are 0

# Check the filtered data
print(males_data[['Age_group', 'Mean Heart Rate']].head())
print(females_data[['Age_group', 'Mean Heart Rate']].head())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Create a figure with subplots
plt.figure(figsize=(14, 6))

# Boxplot for Mean Heart Rate by Age Group for Males
plt.subplot(1, 2, 1)
sns.boxplot(data=males_data, x='Age_group', y='Mean Heart Rate')
plt.title('Mean Heart Rate by Age Group (Male)')
plt.xlabel('Age Group')
plt.ylabel('Mean Heart Rate (BPM)')
plt.grid(axis='y')

# Boxplot for Mean Heart Rate by Age Group for Females
plt.subplot(1, 2, 2)
sns.boxplot(data=females_data, x='Age_group', y='Mean Heart Rate')
plt.title('Mean Heart Rate by Age Group (Female)')
plt.xlabel('Age Group')
plt.ylabel('Mean Heart Rate (BPM)')
plt.grid(axis='y')

# Show the plots
plt.tight_layout()
plt.show()

# Calculate correlation coefficients
if merged_data['Age_group'].dtype == 'int':
    correlation_hr_mean_age_male = males_data[['Mean Heart Rate', 'Age_group']].corr().iloc[0, 1]
    correlation_hr_mean_age_female = females_data[['Mean Heart Rate', 'Age_group']].corr().iloc[0, 1]
    
    print(f"Correlation between Mean Heart Rate and Age Group (Male): {correlation_hr_mean_age_male:.2f}")
    print(f"Correlation between Mean Heart Rate and Age Group (Female): {correlation_hr_mean_age_female:.2f}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Create a figure with subplots
plt.figure(figsize=(14, 6))

# Boxplot for Mean Heart Rate by Age Group for Males
plt.subplot(1, 2, 1)
sns.boxplot(data=merged_data[merged_data['Sex'] == '0'], x='Age_group', y='Mean Heart Rate')
plt.title('Mean Heart Rate by Age Group (Male)')
plt.xlabel('Age Group')
plt.ylabel('Mean Heart Rate (BPM)')
plt.grid(axis='y')

# Boxplot for Mean Heart Rate by Age Group for Females
plt.subplot(1, 2, 2)
sns.boxplot(data=merged_data[merged_data['Sex'] == '1'], x='Age_group', y='Mean Heart Rate')
plt.title('Mean Heart Rate by Age Group (Female)')
plt.xlabel('Age Group')
plt.ylabel('Mean Heart Rate (BPM)')
plt.grid(axis='y')

# Show the plots
plt.tight_layout()
plt.show()

# Calculate correlation coefficients if age groups are numerical
if merged_data['Age_group'].dtype == 'int':
    correlation_hr_mean_age_male = merged_data[merged_data['Gender'] == 'Male'][['Mean Heart Rate', 'Age_group']].corr().iloc[0, 1]
    correlation_hr_mean_age_female = merged_data[merged_data['Gender'] == 'Female'][['Mean Heart Rate', 'Age_group']].corr().iloc[0, 1]
    
    print(f"Correlation between Mean Heart Rate and Age Group (Male): {correlation_hr_mean_age_male:.2f}")
    print(f"Correlation between Mean Heart Rate and Age Group (Female): {correlation_hr_mean_age_female:.2f}")

In [ ]:
# Analyze the relationship between HRV and Age Group
plt.figure(figsize=(12, 6))
sns.boxplot(data=merged_data, x='Age_group', y='HRV')  # Removed palette
plt.title('HRV by Age Group')
plt.xlabel('Age Group')
plt.ylabel('HRV (SDNN)')
plt.grid(axis='y')
plt.show()

# Calculate correlation coefficients if age groups are numerical
if merged_data['Age_group'].dtype == 'int':
    correlation_hrv_age = merged_data[['HRV', 'Age_group']].corr().iloc[0, 1]
    print(f"Correlation between HRV and Age Group: {correlation_hrv_age:.2f}")